In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [7]:
class GaussianLoss(nn.Module):
    def __init__(self, feature_dim=10, cov_dim=10,alpha=0.9, device='cuda'):
        super(GaussianLoss, self).__init__()
        self.alpha = alpha
        self.mean = torch.zeros(cov_dim).cuda()
        self.cov = torch.eye(cov_dim, cov_dim).cuda()
        self.feature_dim = feature_dim
        self.cov_dim = cov_dim
        # self.cuda_num = torch.cuda.current_device()
        self.linear = nn.Linear(feature_dim, cov_dim)
    def forward(self, x):
        # x = (batch, feature_dim)
        x = self.linear(x)
        x = F.normalize(x, dim=1)
        mean_ = (1 - self.alpha) * self.mean + self.alpha * x # (batch, feature_dim)
        x_minus_mean = x - mean_ # (batch, feature_dim)
        # (batch,feature_dim,1) * (batch, 1, feature_dim) -> (batch, feature_dim, feature_dim)
        cov_ = torch.bmm(x_minus_mean.unsqueeze(2), x_minus_mean.unsqueeze(1)) #+ 1e-03 # [256, 128, 128]
        cov_.add_((torch.eye(self.cov_dim)*1e-7).cuda())
        cov_ = (1 - self.alpha) * self.cov + self.alpha * cov_ 
        print(cov_)
        #print(mean_.shape)
        #mean_ = torch.mean(mean_, dim=0).detach()
        #cov_ = torch.mean(cov_, dim=0).detach()
        print(mean_.shape,cov_.shape)
        # loss calculation
        dist = torch.distributions.MultivariateNormal(mean_, covariance_matrix=cov_)
        #print('dist.sample:',dist.sample().shape, dist.sample())
        loss = 0#dist.log_prob(x)
        # change the running mean and running variance
        self.mean = mean_.mean(dim=0).detach() # (1, feature_dim)
        self.cov = cov_.mean(dim=0).detach()
        #loss = torch.log(loss)
        return loss#.mean() #loss

In [8]:
l = GaussianLoss().cuda()

In [9]:
x= torch.randn(15,10).cuda()


In [10]:
l(x)

tensor([[[ 1.0019e-01, -7.4541e-04, -8.2174e-06,  ..., -4.1343e-04,
          -5.8962e-04, -1.3652e-04],
         [-7.4541e-04,  1.0292e-01,  3.2144e-05,  ...,  1.6172e-03,
           2.3064e-03,  5.3402e-04],
         [-8.2174e-06,  3.2144e-05,  1.0000e-01,  ...,  1.7828e-05,
           2.5426e-05,  5.8870e-06],
         ...,
         [-4.1343e-04,  1.6172e-03,  1.7828e-05,  ...,  1.0090e-01,
           1.2792e-03,  2.9619e-04],
         [-5.8962e-04,  2.3064e-03,  2.5426e-05,  ...,  1.2792e-03,
           1.0182e-01,  4.2241e-04],
         [-1.3652e-04,  5.3402e-04,  5.8870e-06,  ...,  2.9619e-04,
           4.2241e-04,  1.0010e-01]],

        [[ 1.0129e-01,  1.3790e-03, -1.3187e-04,  ...,  5.5309e-04,
           1.1285e-03,  2.8353e-04],
         [ 1.3790e-03,  1.0147e-01, -1.4054e-04,  ...,  5.8947e-04,
           1.2027e-03,  3.0218e-04],
         [-1.3187e-04, -1.4054e-04,  1.0001e-01,  ..., -5.6367e-05,
          -1.1500e-04, -2.8896e-05],
         ...,
         [ 5.5309e-04,  5

0

In [2]:
test = torch.distributions.MultivariateNormal(torch.zeros(10), torch.eye(10))

In [3]:
test.sample()

tensor([ 0.3162,  0.7397, -1.4405,  0.7338, -0.0237,  0.7221,  0.4206, -0.9367,
        -0.9734,  0.4955])

In [6]:
torch.zeros(10).cuda()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')